# Mistral मॉडेल्ससह बांधणी

## परिचय

या धड्यात आपण पाहणार आहोत:
- वेगवेगळ्या Mistral मॉडेल्सचा अभ्यास
- प्रत्येक मॉडेलसाठी वापराचे प्रकार आणि परिस्थिती समजून घेणे
- प्रत्येक मॉडेलच्या खास वैशिष्ट्ये दाखवणारे कोड नमुने


## मिस्त्राल मॉडेल्स

या धड्यात, आपण ३ वेगवेगळ्या मिस्त्राल मॉडेल्सचा अभ्यास करू: **मिस्त्राल लार्ज**, **मिस्त्राल स्मॉल** आणि **मिस्त्राल नेमो**.

ही सर्व मॉडेल्स Github Model मार्केटप्लेसवर मोफत उपलब्ध आहेत. या नोटबुकमधील कोड या मॉडेल्सचा वापर करून कोड चालवेल. Github Models वापरून [AI मॉडेल्ससह प्रोटोटायपिंग](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) कसे करावे याबद्दल अधिक माहिती येथे दिली आहे.


## मिस्त्राल लार्ज २ (२४०७)
मिस्त्राल लार्ज २ सध्या मिस्त्रालकडून उपलब्ध असलेला प्रमुख मॉडेल आहे आणि हे खास एंटरप्राइज वापरासाठी डिझाइन केले आहे.

हे मॉडेल मूळ मिस्त्राल लार्जपेक्षा पुढे आहे कारण यात खालील सुधारणा आहेत:
- मोठे कॉन्टेक्स्ट विंडो - १२८k विरुद्ध ३२k
- गणित आणि कोडिंग टास्कमध्ये अधिक चांगली कामगिरी - सरासरी अचूकता ७६.९% विरुद्ध ६०.४%
- बहुभाषिक कामगिरीत वाढ - यात समाविष्ट भाषा: इंग्रजी, फ्रेंच, जर्मन, स्पॅनिश, इटालियन, पोर्तुगीज, डच, रशियन, चिनी, जपानी, कोरियन, अरबी आणि हिंदी.

या वैशिष्ट्यांसह, मिस्त्राल लार्ज खालील गोष्टींमध्ये उत्कृष्ट आहे:
- *रिट्रीव्हल ऑगमेंटेड जनरेशन (RAG)* - मोठ्या कॉन्टेक्स्ट विंडोमुळे
- *फंक्शन कॉलिंग* - या मॉडेलमध्ये नेटिव्ह फंक्शन कॉलिंग आहे, ज्यामुळे बाह्य टूल्स आणि API शी सहजपणे एकत्रीकरण करता येते. हे कॉल्स एकाच वेळी किंवा एकामागोमाग एक अशा अनुक्रमे पद्धतीने करता येतात.
- *कोड जनरेशन* - हे मॉडेल Python, Java, TypeScript आणि C++ कोड जनरेशनमध्ये उत्कृष्ट आहे.


या उदाहरणात, आपण Mistral Large 2 वापरून एका मजकूर दस्तऐवजावर RAG पॅटर्न चालवत आहोत. प्रश्न कोरियन भाषेत लिहिलेला आहे आणि लेखकाने कॉलेजमध्ये जाण्यापूर्वी केलेल्या क्रियाकलापांबद्दल विचारतो.

हे Cohere Embeddings Model वापरते जेणेकरून मजकूर दस्तऐवजाचे आणि प्रश्नाचे embeddings तयार करता येतील. या नमुन्यासाठी, faiss या Python पॅकेजचा वापर व्हेक्टर स्टोअर म्हणून केला आहे.

Mistral मॉडेलला पाठवलेला prompt यात प्रश्न आणि प्रश्नाशी साधर्म्य असलेले retrieved chunks दोन्ही समाविष्ट असतात. त्यानंतर मॉडेल नैसर्गिक भाषेत उत्तर देते.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small हा Mistral कुटुंबातील आणखी एक मॉडेल आहे, जे प्रीमियर/एंटरप्राइज श्रेणीमध्ये येते. नावावरूनच कळते की, हे मॉडेल एक Small Language Model (SLM) आहे. Mistral Small वापरण्याचे फायदे म्हणजे:
- खर्चात बचत - Mistral Large आणि NeMo सारख्या Mistral LLMs च्या तुलनेत ८०% किंमत कमी
- कमी विलंब - Mistral च्या LLMs पेक्षा जलद प्रतिसाद
- लवचिक - आवश्यक संसाधनांवर कमी निर्बंधांसह विविध वातावरणात तैनात करता येते

Mistral Small खालील गोष्टींसाठी उत्तम आहे:
- मजकूर आधारित कामे जसे की सारांश तयार करणे, भावना विश्लेषण आणि भाषांतर
- खर्च कार्यक्षमतेमुळे ज्या अनुप्रयोगांमध्ये वारंवार विनंत्या केल्या जातात
- कमी विलंब असलेली कोड संबंधित कामे जसे की कोड पुनरावलोकन आणि कोड सुचना


## मिस्त्राल स्मॉल आणि मिस्त्राल लार्जची तुलना

मिस्त्राल स्मॉल आणि लार्ज यांच्यातील विलंब (latency) कसा वेगळा आहे हे दाखवण्यासाठी खालील सेल्स चालवा.

तुम्हाला प्रत्युत्तर वेळेत साधारण ३-५ सेकंदांचा फरक दिसेल. तसेच, एकाच प्रॉम्प्टवर प्रत्युत्तराची लांबी आणि शैली यामध्येही फरक जाणवेल.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

या धड्यात चर्चा केलेल्या इतर दोन मॉडेल्सच्या तुलनेत, Mistral NeMo हे एकमेव मोफत मॉडेल आहे ज्याला Apache2 परवाना आहे.

हे Mistral या आधीच्या ओपन सोर्स LLM, Mistral 7B चे अपग्रेड मानले जाते.

NeMo मॉडेलच्या काही इतर वैशिष्ट्यांमध्ये:

- *अधिक कार्यक्षम टोकनायझेशन:* हे मॉडेल सामान्यतः वापरल्या जाणाऱ्या tiktoken ऐवजी Tekken tokenizer वापरते. त्यामुळे अधिक भाषांमध्ये आणि कोडमध्ये चांगली कामगिरी मिळते.

- *फाइनट्यूनिंग:* बेस मॉडेल फाइनट्यूनिंगसाठी उपलब्ध आहे. त्यामुळे ज्या वापराच्या परिस्थितीत फाइनट्यूनिंग आवश्यक असेल, तिथे अधिक लवचिकता मिळते.

- *नेटिव फंक्शन कॉलिंग* - Mistral Large प्रमाणे, या मॉडेलला फंक्शन कॉलिंगवर प्रशिक्षण दिले गेले आहे. त्यामुळे हे ओपन सोर्स मॉडेल्समध्ये असे करणारे पहिले मॉडेल्सपैकी एक ठरते.


## मिस्त्राल नेमो

या धड्यात चर्चा केलेल्या इतर दोन मॉडेल्सच्या तुलनेत, मिस्त्राल नेमो हे एकमेव मोफत मॉडेल आहे ज्याला Apache2 परवाना आहे.

हे मिस्त्रालच्या आधीच्या ओपन सोर्स LLM, मिस्त्राल 7B चे अपग्रेड मानले जाते.

नेमो मॉडेलच्या काही इतर वैशिष्ट्ये:

- *अधिक कार्यक्षम टोकनायझेशन:* हे मॉडेल सामान्यतः वापरल्या जाणाऱ्या tiktoken ऐवजी Tekken टोकनायझर वापरते. यामुळे अधिक भाषांमध्ये आणि कोडमध्ये चांगली कार्यक्षमता मिळते.

- *फाइनट्यूनिंग:* बेस मॉडेल फाइनट्यूनिंगसाठी उपलब्ध आहे. त्यामुळे ज्या वापराच्या परिस्थितीत फाइनट्यूनिंग आवश्यक असेल, तिथे अधिक लवचिकता मिळते.

- *नेटिव फंक्शन कॉलिंग* - मिस्त्राल लार्ज प्रमाणेच, या मॉडेलला फंक्शन कॉलिंगवर प्रशिक्षण दिले आहे. त्यामुळे हे ओपन सोर्स मॉडेल्समध्ये असे करणारे पहिले मॉडेल्सपैकी एक आहे.


### टोकनायझर्सची तुलना

या उदाहरणात, आपण पाहूया की Mistral NeMo टोकनायझेशन कसे हाताळते आणि ते Mistral Large पेक्षा कसे वेगळे आहे.

दोन्ही उदाहरणांमध्ये एकच प्रॉम्प्ट वापरला आहे, पण तुम्ही पाहू शकता की NeMo मध्ये Mistral Large पेक्षा कमी टोकन्स परत मिळतात.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## शिक्षण इथेच थांबत नाही, प्रवास सुरू ठेवा

हे धडा पूर्ण केल्यानंतर, आमच्या [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ला भेट द्या आणि तुमचे Generative AI ज्ञान आणखी वाढवा!



---

**अस्वीकरण**:  
हे दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केले आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानावा. अत्यावश्यक माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थ लावण्यास आम्ही जबाबदार राहणार नाही.
